<a href="https://colab.research.google.com/github/jiahaom/MLOps/blob/main/1_Data_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Validation

## 1.1 Data Quality
> Data is the First Class Citizen for ML.

ℹ️ Data quality is important for a data pipeline in MLops because it affects the performance and accuracy of the machine learning models that are built and deployed. Ensuring high data quality is an important step in the MLops process to ensure that the models are reliable and accurate.

❌ Poor quality data can lead to inaccurate or biased models, which can result in poor decision-making and poor performance in production. Additionally, poor data quality can also cause delays and additional costs in the development and deployment of machine learning models. 

✅ By generating and visualizing data statistics, it is possible to get a better understanding of the data and identify potential problems early on. This can help to improve the performance and accuracy of the machine learning models that are built and deployed.

## 1.2 TensorFlow Data Validation 
ℹ️ TensorFlow Data Validation (TFDV) is a library provided by TensorFlow that can be used in MLOps to validate and analyze large datasets before they are used to train machine learning models. 

✅ By using TFDV, data scientists and engineers can detect and correct data quality issues early on, which can help to improve the performance and accuracy of the models. Additionally, it can also be used to detect data drift, monitor data distribution changes and detect data skew, which can help to ensure that the models continue to perform well in production.
The library provides tools for:

- Schema Generation: It creates a schema for a dataset based on the data types and statistics of the features.
- Schema Comparison: It can compare two schemas and identify differences between them, which can help to detect changes in the data distribution.
- Data Validation: It can check if the data conforms to the schema, including checking for missing values, data types, and feature ranges, and can also detect and notify about data outliers.
- Data Statistics: It can generate statistics for the data, such as mean, standard deviation, and histograms, which can be used to visualize the data and identify potential problems.
- Anomaly Detection: It can detect anomalies in the data by comparing the statistics of the data to the statistics of a previous version of the data.

🖥️ To use TFDV in Google Colab, you will need to first install the TensorFlow Data Validation library. We can do this by running the following command in a Google Colab notebook cell:
> Hint: TFDV doesn't support Apple silicon currently (Jan 2023)

In [2]:
# !pip install tensorflow_data_validation

Once the library is installed, we can start using it to validate and analyze our data.

In [3]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

print('TFDV Version: {}'.format(tfdv.__version__))
print('Tensorflow Version: {}'.format(tf.__version__))

TFDV Version: 1.12.0
Tensorflow Version: 2.11.0


## 1.3 Load the Dataset


The dataset we used is about MOOC dropout prediction, which includes followed features:
- user_log_num
- course_log_num
- take_course_num
- take_user_num
- log_num
- server_nagivate
- server_access
- server_problem
- browser_access
- browser_problem
- browser_page_close
- browser_video
- server_discussion
- server_wiki
- chapter_count
- sequential_count
- video_count
- problem_count
- discussion_count
- taked_course_chapter_amount
- taked_course_video_amount
- taked_course_discussion_amount

In [4]:
import pandas as pd

data = pd.read_csv('Mooc.csv',index_col = 'Unnamed: 0')

data.head()

,user_log_num,course_log_num,take_course_num,take_user_num,log_num,server_nagivate,server_access,server_problem,browser_access,browser_problem,...,server_wiki,chapter_count,sequential_count,video_count,problem_count,discussion_count,taked_course_chapter_amount,taked_course_video_amount,taked_course_discussion_amount,Success
0,1029,878119,6,10322,314,25,86,8,21,79,...,0,13,80,29,87,0,13,56,0,0
1,839,148500,3,1481,288,14,45,3,34,135,...,0,9,60,9,138,26,12,113,129,0
2,289,878119,2,10322,99,15,64,1,0,5,...,0,11,41,4,6,0,13,56,0,0
3,633,148500,1,1481,633,30,106,32,120,138,...,0,14,197,82,170,34,12,113,129,0
4,23,334927,1,7775,23,5,12,0,0,2,...,0,1,9,2,2,0,14,57,0,0


## 1.4 Split the Dataset
In order to ensure the models are reliable and accurate, we split our dataset into 3 parts: Training (60%), Validation (20%) and Serving (20%).

In [5]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, test_size=0.2, random_state = 0)
train_data, serve_data = train_test_split(train_data, test_size=0.25, random_state = 0)
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
serve_data = serve_data.drop(['Success'], axis=1).reset_index(drop=True)
print('Training dataset has {} records\nValidation dataset has {} records\nServing dataset has {} records'.format(len(train_data),len(val_data),len(serve_data)))

Training dataset has 57860 records
Validation dataset has 19287 records
Serving dataset has 19287 records


## 1.5 Generate Training Data Statistics

There are two ways to generate training data statistics:

### 1.5.1 From Pandas.DataFrame


In [6]:
# feature_allowlist indicates which feature could be calculated for statistics.
stats_options = tfdv.StatsOptions(feature_allowlist = list(data.columns))

# Review allowed features
for feature in stats_options.feature_allowlist:
    print(feature)

user_log_num
course_log_num
take_course_num
take_user_num
log_num
server_nagivate
server_access
server_problem
browser_access
browser_problem
browser_page_close
browser_video
server_discussion
server_wiki
chapter_count
sequential_count
video_count
problem_count
discussion_count
taked_course_chapter_amount
taked_course_video_amount
taked_course_discussion_amount
Success


### 1.5.2 From CSV file

In [7]:
stats_csv = tfdv.generate_statistics_from_csv(data_location='Mooc.csv', delimiter=',')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## 1.6 Visualize Training Data Statistics

In [8]:
train_stats = tfdv.generate_statistics_from_dataframe(train_data)
tfdv.visualize_statistics(train_stats)

## 1.7 Generate Training Data Schema 

ℹ️ A data schema is a blueprint or a set of rules that defines the structure, organization, and format of a dataset. It describes the types of data that are stored in the dataset, the relationships between different data elements, and the constraints on the data values.

✅ TFDV provides a method called `generate_schema()` that can automatically create a schema based on the data types and statistics of the features in a dataset. The schema is represented as a Schema object, which can be used to validate and analyze the data. For example, by comparing the statistics and schema of different versions of the data to detect data drift, or by comparing the statistics of different classes to detect data skew.

❌ Keep in mind that, schema generation is not a replacement for traditional data cleaning and preprocessing steps, but it can be a useful tool for identifying data quality issues, and for monitoring the data pipeline in MLops.

In [9]:
train_schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(train_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'user_log_num',INT,required,,-
'course_log_num',INT,required,,-
'take_course_num',INT,required,,-
'take_user_num',INT,required,,-
'log_num',INT,required,,-
'server_nagivate',INT,required,,-
'server_access',INT,required,,-
'server_problem',INT,required,,-
'browser_access',INT,required,,-


## 1.8 Recognize and Fix DataData Problems

### 1.8.1 Compare Datasets

ℹ️ Schema skew refers to a situation where the schema of a dataset changes over time, leading to inconsistencies or inaccuracies in the data. This can happen for various reasons, such as changes in data sources, data processing pipelines, or data storage systems.

❌ Schema skew can cause a variety of problems in data processing and analysis, such as:

1. Inconsistencies in the data that can lead to incorrect or incomplete results.
1. Difficulty in understanding the data, as the schema is not consistent across the dataset.
1. Difficulty in integrating and merging data from different sources, as the schema is not consistent across the dataset.
1. Difficulty in implementing data governance and data quality control, as the schema is not consistent across the dataset.

✅  It's important to keep monitoring the data pipeline, validate the data against the schema, and update the schema accordingly. This can be done by using tools such as TensorFlow Data Validation (TFDV) which can detect and notify about schema skew by comparing the statistics and schema of different versions of the data.

In [10]:
print(f"First feature of Train: {train_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature of Train: {train_stats.datasets[0].features[-1].path.step[0]}")
print(f"Number of features used: {len(train_stats.datasets[0].features)}")
print(f"Number of examples used: {train_stats.datasets[0].num_examples}\n")



val_stats = tfdv.generate_statistics_from_dataframe(val_data, stats_options)
print(f"First feature of Val: {val_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature of Val: {val_stats.datasets[0].features[-1].path.step[0]}")
print(f"Number of features used: {len(val_stats.datasets[0].features)}")
print(f"Number of examples used: {val_stats.datasets[0].num_examples}")

test_stats = tfdv.generate_statistics_from_dataframe(serve_data, stats_options)

First feature of Train: user_log_num
Last feature of Train: Success
Number of features used: 23
Number of examples used: 57860

First feature of Val: user_log_num
Last feature of Val: Success
Number of features used: 23
Number of examples used: 19287


In [11]:
# Generate evaluation dataset statistics
val_info = tfdv.generate_statistics_from_dataframe(val_data)

# Compare training with evaluation
tfdv.visualize_statistics(
    rhs_statistics=train_stats,rhs_name='TRAIN',
    lhs_statistics=val_stats, lhs_name='VAL') 

### 1.8.2 Validation anomalies

In [12]:
# Anomalies Detection and Visualization
outlier = tfdv.validate_statistics(statistics=val_stats, schema=train_schema)
tfdv.display_anomalies(outlier)

We'd like to see `No anomalies found.` after anomalies detection. Otherwise, we need to fix validation anomalies in the schema. For example:

  > - `Feature name: Skills`
  > - `Anomaly short description: Unexpected string values`
  > - `Unexpected string values: Examples contain values missing from the schema: Accounting (<1%).` 

which indicates validation data has records with `Accounting` for the features `Skills` that were not included in the schema generated from the training data. 

We can fix this by adding the new values that exist in the validation dataset to the domain of these features.

```Python
skills_domain = tfdv.get_domain(train_schema, 'Skills') 
skills_domain.value.append("Accounting")

calculate_and_display_anomalies(val_stats, schema=train_schema)
```

### 1.8.3 Serving anomalies
Likewise, we'd like to see `No anomalies found.` after anomalies detection.


In [13]:
options = tfdv.StatsOptions(schema=train_schema, 
                            infer_type_from_schema=True, 
                            feature_allowlist=list(data.columns))

serve_stats = tfdv.generate_statistics_from_dataframe(serve_data, stats_options=options)
outlier = tfdv.validate_statistics(statistics=serve_stats, schema=train_schema)
tfdv.display_anomalies(outlier)

,Anomaly short description,Anomaly long description
Feature name,,
'Success',Column dropped,Column is completely missing


As Anomaly short description shows, we dropped 'Success'/labels column from serving dataset. Hence, we can let TFDV to ignore the anomaly.

In [14]:
# All features are by default in both TRAINING and SERVING environments.
train_schema.default_environment.append('TRAINING')
train_schema.default_environment.append('SERVING')

tfdv.get_feature(train_schema, 'Success').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(serve_stats, train_schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

There is a chance to see followed example:

  > - `Feature name: Skills`
  > - `Anomaly short description: Unexpected string values`
  > - `Unexpected string values: Examples contain values missing from the schema: Accounting (<1%).` 

This is because we have a restrict detection constraints. Usually we can solve the error by relaxing the rule.

```Python
# relax to match 90% of the domain
skills = tfdv.get_feature(schema=train_schema, 'Skills')
skills.distribution_constraints.min_domain_mass = 0.9 

calculate_and_display_anomalies(serving_stats, schema=train_schema)
```

### 1.8.4 Data Drift and Skew

1. Data Drift

  ℹ️ Data drift occurs when the distribution of the data used to train a machine learning model differs from the distribution of the data that the model encounters in production. 

  ❌ This can lead to a decrease in model performance and accuracy. 

  🤔 Data drift can be caused by changes in the data generating process or by the accumulation of stale data.

1. Data Skew

  ℹ️ Data skew occurs when the data used to train a machine learning model is not evenly distributed among the different classes or categories. 
  
  ❌ This can lead to a bias in the model towards the more frequent classes, resulting in poor performance on less frequent classes. 
  
  🤔 Data skew can be caused by imbalanced data collection or by data preprocessing steps that do not properly balance the data.

In [15]:
# Calculate skew for the course_log_num feature
math_score = tfdv.get_feature(train_schema, 'course_log_num')
math_score.skew_comparator.infinity_norm.threshold = 0.01 # domain knowledge helps to determine this threshold

# Calculate drift for the taked_course_video_amount feature
reading_score = tfdv.get_feature(train_schema, 'taked_course_video_amount')
reading_score.drift_comparator.infinity_norm.threshold = 0.01 # domain knowledge helps to determine this threshold

# Calculate anomalies
skew_drift_anomalies = tfdv.validate_statistics(train_stats, train_schema,
                                               previous_statistics=val_stats,
                                                serving_statistics=serve_stats)

# Display anomalies
tfdv.display_anomalies(skew_drift_anomalies)

✅ Ideally, we could see `No anomalies found.` as aboved. 

❌ Otherwise, we could know how the Linfty distance between training and serving is, and decide whether accept a new  threshold or not. If the anomaly truly indicates a skew and drift, then further investigation is necessary as this could have a direct impact on model performance.

### 1.8.5 Data Slices

ℹ️ Data slicing is the process of dividing a dataset into subsets based on specific features or attributes. By slicing the data, we can analyze the statistics and schema of different subsets of the data and compare them to identify any potential biases.

✅ For example, if we are concerned about bias in a dataset based on `take_course_num`, we could slice the data by `take_course_num` and compare the statistics and schema for each number's subsets. If there are significant differences in the statistics or schema for the subsets, it may indicate that the data is biased.

❌ Data slicing is not a replacement for traditional methods of bias detection and mitigation, but it can be a useful tool to help identify potential biases in the data. When using data slicing, it's important to keep in mind that bias can be introduced in many ways and it should be used in conjunction with other methods such as fairness metrics, human review, and domain expertise.

In [18]:
from tensorflow_data_validation.utils import slicing_util
# Slice function for the `take_course_num` feature
slice_fn = slicing_util.get_feature_value_slicer(features={'take_course_num': None})

In [20]:
slice_options = tfdv.StatsOptions(schema=train_schema,
                                  slice_functions=[slice_fn],
                                  infer_type_from_schema=True,
                                  feature_allowlist=list(data.columns))
# `tfdv.generate_statistics_from_csv` only work with CSV
data.to_csv('temp.csv')

# Slice stats
sliced_stats = tfdv.generate_statistics_from_csv('temp.csv', stats_options=slice_options)


In [24]:
# Split the dataset
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList

slice_datasets = []
for dataset in sliced_stats.datasets:
  proto_list = DatasetFeatureStatisticsList()
  proto_list.datasets.extend([dataset])
  slice_datasets.append(proto_list)

In [45]:
print(f'Statistics generated for:\n')
for i,j in enumerate(slice_datasets):
  print (i, j.datasets[0].name)

Statistics generated for:

0 All Examples
1 take_course_num_6
2 take_course_num_3
3 take_course_num_2
4 take_course_num_1
5 take_course_num_4
6 take_course_num_7
7 take_course_num_14
8 take_course_num_5
9 take_course_num_8
10 take_course_num_12
11 take_course_num_9
12 take_course_num_11
13 take_course_num_10
14 take_course_num_15
15 take_course_num_25
16 take_course_num_13
17 take_course_num_21
18 take_course_num_20
19 take_course_num_17
20 take_course_num_16
21 take_course_num_18
22 take_course_num_19
23 take_course_num_27


✅ Afterwards, we can compare the statistics for `take_course_num_27 (index=23)` with `all examples (index=0)` easily:


In [51]:
tfdv.visualize_statistics(
    rhs_statistics=slice_datasets[0],rhs_name='ALL_EXAMPLES',
    lhs_statistics=slice_datasets[23], lhs_name='SLICED_DATA') 

✅ Finally, we can use several tools from TFDV that enable us to recognized and fix some data problems.